# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [3]:
import pandas as pd
import numpy as np
from scipy import stats

In [4]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [5]:
# set some variables with proportions to use in problem
black = data[data.race == 'b'].race.count()
white = data[data.race=='w'].race.count()
black_yes = sum(data[data.race=='b'].call)
white_yes = sum(data[data.race=='w'].call)
print ('Black Callback', black_yes, 'White Population', white)
print ('White Callback', white_yes, 'Black Population', black)


Black Callback 157.0 White Population 2435
White Callback 235.0 Black Population 2435


1)  A proportions z-test is the most appropriate for this problem. The Central Limit Theorem here does apply, as the methodology for assigning black and white names to resumes was random, and the number of resumes sent for both black and white-sounding names exceed 30. 

2) 
Null Hypothesis:  The proportion of callbacks with white sounding names is less than or equal to the proportion of callbacks with black sounding names.
   
Alternative Hypothesis: The proportion of callbacks with black sounding names is less than the proportion of callbacks with white-sounding names 

3) Compute margin of error, confidence interval, and p-value.

In [8]:
# import proportions z test from statsmodels
from statsmodels.stats.proportion import proportions_ztest
import numpy as np
# create numpy array of callbacks for white and black resumes
count = np.array([int(white_yes), int(black_yes)])
# create a numpy array of total number of resumes sent for white and black
nobs = np.array([int(white), int(black)])
# run proportions z test assuming null hypothesis is true, hence "larger" as argument for alternative
z, p = proportions_ztest(count, nobs, alternative = 'larger' )
print ("p-value: {} is smaller than alpha .05, so reject null hypothesis".format(p))

p-value: 1.9919434187925383e-05 is smaller than alpha .05, so reject null hypothesis


In [19]:
from statsmodels.stats import proportion
import math

white_proportion = white_yes/ int(white)
black_proportion = black_yes/ int(black)


# compute standard error of each proportion
SE_white = math.sqrt((white_proportion * (1 - white_proportion))/(white))
SE_black = math.sqrt((black_proportion * (1 - black_proportion))/(black))

# compute standard error for difference
SE_difference = math.sqrt(SE_white **2 + SE_black **2)
# z-score corresponds to 95% level of confidence
z = 1.96
# absolute difference between white and black proportions
difference = abs(white_proportion - black_proportion)
# calculate upper bound of confidence interval
upper = (difference) + z * SE_difference
# calculate lower bound of confidence interval
lower = (difference) - z * SE_difference
# calculate margin of error
margin_of_error = z * SE_difference
print ("Margin of Error:", round(margin_of_error, 4))
print ("Lower Bound:", round(lower, 4), "\n" "Upper Bound:", round(upper, 4))
print ("Confidence Interval:", lower, "-", upper )


Margin of Error: 0.0153
Lower Bound: 0.0168 
Upper Bound: 0.0473
Confidence Interval: 0.016777447859559147 - 0.047288260559332024


4) In the case of whether or not the data suggest racial discrimination in job call-backs based on racially coded resume names, the p-value of 0.00002 at a .05 level of significance produced through a two-sided proportions z-test leads to a rejection of the null hypothesis; that the proportion of callbacks for white names by the true population of employers is less than or equal to the proportion of callbacks for black names. In other words, there is a .002% chance that the null hypothesis holds true for the true population given the data collected. The alternative hypothesis - that black resumes receive less callbacks than white resumes - should be accepted. A confidence interval calculated for the data suggests that 95% of the time, the difference between the proportion of white resume and black resume callbacks will be between 1.7% and 4.7%, with a chance that the difference between the two will be 1.5% below or above this interval. 

5) While the results of a z-test investigating a significant difference in resume callbacks can help answer the question of whether or not racial discrimination exists in the marketplace, merely performing a z-test is not enough to say that this is the most important factor in callback success. The analysis analysis could be ammended by performing a multivariate linear regression across several variables defined as important. This will allow for somewhat of a control in determining which variable has the greatest statistically significant correlation with callbak success. 